In [1]:
import geopandas as gpd
import rasterio as rio
import rasterstats as rs
from rasterstats import zonal_stats
import numpy as np
OA_path = r"C:\Studie\SSML\project\statistical-gis-boundaries-london\ESRI\OA_2011_London_gen_MHW.shp"
temp_path = r"C:\Studie\SSML\project\temperature\tasmonthly\subset_data.tif"
OA_gdf = gpd.read_file(OA_path,
                       engine = 'pyogrio',
                       use_arrow = True)
dataset = rio.open(temp_path)

c:\Users\Gebruiker\anaconda3\envs\geospatial\Lib\site-packages\pyogrio\raw.py:329: RuntimeWarning: C:\Studie\SSML\project\statistical-gis-boundaries-london\ESRI\OA_2011_London_gen_MHW.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  table = reader.read_all()


In [2]:
OA_gdf.head()

,OA11CD,LSOA11CD,MSOA11CD,WD11CD_BF,WD11NM_BF,LAD11CD,LAD11NM,RGN11CD,RGN11NM,LSOA11NM,MSOA11NM,USUALRES,HHOLDRES,COMESTRES,POPDEN,HHOLDS,AVHHOLDSZ,geometry
0,E00023264,E01004612,E02000954,E05000626,Tooting,E09000032,Wandsworth,E12000007,London,Wandsworth 032C,Wandsworth 032,462,459,3,115.2,143,3.2,"POLYGON ((527635.452 171810.058, 527495.957 17..."
1,E00003359,E01000692,E02000128,E05000111,Chislehurst,E09000006,Bromley,E12000007,London,Bromley 002D,Bromley 002,269,259,10,36.7,133,1.9,"POLYGON ((543581.516 171132.486, 543704 171022..."
2,E00023266,E01004615,E02000956,E05000626,Tooting,E09000032,Wandsworth,E12000007,London,Wandsworth 034B,Wandsworth 034,277,277,0,183.4,133,2.1,"POLYGON ((526613.539 171839.165, 526546.84 171..."
3,E00020264,E01004027,E02000809,E05000548,Riverside,E09000028,Southwark,E12000007,London,Southwark 003E,Southwark 003,415,415,0,96.1,191,2.2,"POLYGON ((533563.521 179695.146, 533351.118 17..."
4,E00023263,E01004613,E02000955,E05000626,Tooting,E09000032,Wandsworth,E12000007,London,Wandsworth 033D,Wandsworth 033,304,304,0,165.2,131,2.3,"POLYGON ((527822 172020, 527979 171903.001, 52..."


In [3]:
shape_o = None
for i in range(1, dataset.count + 1):
    band = dataset.read(i)
    shape_n = band.shape
    if shape_o is not None:
        if shape_n != shape_o:
            print(shape_n, shape_o)
    shape_o = band.shape
shape_o

(45, 58)

In [4]:
print(dataset.count, dataset.width, dataset.height, dataset.crs)


168 58 45 EPSG:27700


In [5]:
old_profile = dataset.profile
old_profile

{'driver': 'GTiff', 'dtype': 'float64', 'nodata': 1e+20, 'width': 58, 'height': 45, 'count': 168, 'crs': CRS.from_epsg(27700), 'transform': Affine(1000.0, 0.0, 504000.0,
       0.0, 1000.0, 156000.0), 'blockysize': 1, 'tiled': False, 'interleave': 'pixel'}

In [ ]:
def flip_tif(path):
    dataset = rio.open(path)
    old_tif = dataset
    old_profile = dataset.profile
    new_profile = old_profile
    new_profile['transform'] = rio.Affine(1000.0, 0.0, 504000.0, 0.0, -1000.0, -156000.0)
    new_path = f"{path[:-4]}_flipped.tif"
    with rio.open(new_path, "w", **new_profile) as dest:
        for i in range(1, dataset.count + 1):
            new_array_tif = old_tif.read(i)
            new_array_tif = np.fliplr(np.flip(new_array_tif))
            dest.write(new_array_tif, indexes=i)

In [6]:
#testing fix from: https://stackoverflow.com/questions/62010050/from-zonal-stats-i-get-this-error-valueerror-width-and-height-must-be-0
old_temp_tif = dataset
new_profile = old_profile
new_profile['transform'] = rio.Affine(1000.0, 0.0, 504000.0, 0.0, -1000.0, -156000.0)

In [7]:
new_path = f"{temp_path[:-4]}_flipped.tif"
with rio.open(new_path, "w", **new_profile) as dest:
    for i in range(1, dataset.count + 1):
        new_temp_array_tif = old_temp_tif.read(i)
        new_tif_array = np.fliplr(np.flip(new_temp_array_tif))
        dest.write(new_tif_array, indexes=i)

In [8]:
flipped_ds = rio.open(new_path)

In [9]:
max = 0
for i in range(1, flipped_ds.count + 1):
    max_new = flipped_ds.read(i).max()
    print(max_new)
    if max_new > max and max_new < 1e10:
        max = max_new
        print(max, i)
    

1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+20
1e+2

In [10]:
flipped_ds.read(38)

array([[2.78708279, 2.7955496 , 2.93060911, ..., 2.75186162, 2.83123465,
        2.72475123],
       [2.76577745, 2.9673097 , 3.01595511, ..., 2.77577164, 2.89057799,
        2.79708176],
       [2.79340907, 2.90850044, 3.10803138, ..., 2.84834825, 2.94683165,
        2.85079144],
       ...,
       [3.66568706, 3.67542995, 3.65031714, ..., 2.57750541, 2.62115677,
        2.63442693],
       [3.65790233, 3.64049001, 3.59605733, ..., 2.63726452, 2.70493316,
        2.71745864],
       [3.62740098, 3.56881091, 3.49146338, ..., 2.59362541, 2.74358006,
        2.63421108]])

In [11]:
max = 0
for i in range(1, dataset.count + 1):
    max_new = dataset.read(i).max()
    if max_new > max and max_new < 1e20:
        max = max_new
    
    print(max, i)
        


0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
0 33
0 34
0 35
0 36
0 37
0 38
0 39
0 40
0 41
0 42
0 43
0 44
0 45
0 46
0 47
0 48
0 49
0 50
0 51
0 52
0 53
0 54
0 55
0 56
0 57
0 58
0 59
0 60
0 61
0 62
0 63
0 64
0 65
0 66
0 67
0 68
0 69
0 70
0 71
0 72
0 73
0 74
0 75
0 76
0 77
0 78
0 79
0 80
0 81
0 82
0 83
0 84
0 85
0 86
0 87
0 88
0 89
0 90
0 91
0 92
0 93
0 94
0 95
0 96
0 97
0 98
0 99
0 100
0 101
0 102
0 103
0 104
0 105
0 106
0 107
0 108
0 109
0 110
0 111
0 112
0 113
0 114
0 115
0 116
0 117
0 118
0 119
0 120
0 121
0 122
0 123
0 124
0 125
0 126
0 127
0 128
0 129
0 130
0 131
0 132
0 133
0 134
0 135
0 136
0 137
0 138
0 139
0 140
0 141
0 142
0 143
0 144
0 145
0 146
0 147
0 148
0 149
0 150
0 151
0 152
0 153
0 154
0 155
0 156
0 157
0 158
0 159
0 160
0 161
0 162
0 163
0 164
0 165
0 166
0 167
0 168


In [ ]:
stats_per_band = {}
for band in range(1, flipped_ds.count + 1):
    stats = zonal_stats(OA_path, new_path, band = band)
    stats_per_band[band] = stats